In [39]:
from Research_Final.DatasetLoader_mine2 import DatasetLoader
import os
import numpy as np

In [40]:
parent = 'multipletimeseriesforecasting/preprocessed-data/'

path = os.path.join(parent, 'SanFranciscoTraffic/traffic_weekly_dataset.tsf')

dl = DatasetLoader('SanFranciscoTraffic', path)

matrix_1_original = dl.dataset_load_and_preprocess("original")
matrix_1_norm = dl.dataset_load_and_preprocess("normalized")
print(matrix_1_norm.shape)

---------------------------------------------------
Dataset name:  SanFranciscoTraffic
Dataset contains missing values?  False
--------------------------
---------------------------------------------------
Dataset name:  SanFranciscoTraffic
Dataset contains missing values?  False
--------------------------
(862, 104)


In [41]:
sample_len = int(matrix_1_norm.shape[1]*(0.3))
print("Sample_len total", sample_len)

np.random.seed(1)
sets = np.random.choice(matrix_1_norm.shape[1]-40, int(sample_len), replace=False)
sets_training, sets_testing = sets[:int(len(sets)*.8)], sets[int(len(sets)*.8):]
print(sets_training, sets_testing)

Sample_len total 31
[24 39 52 27 44  2 21 62 41 50 38 54 35 26 57 49  3 40 36 10 56 19 34 60] [17 55 53 48 33 51 31]


## Regular Multimodel HD with time series encoding(permutation)

In [7]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

# Note: this example requires the torchmetrics library: https://torchmetrics.readthedocs.io
import torchmetrics
from tqdm import tqdm

from torchhd import functional
from torchhd import embeddings
from torchhd import cos_similarity

from scipy.special import softmax

import time

d=10000
models = 8

# Model based on RegHD application for Single model regression -> No comparing which cluster
class MultiModel_With_NonLinear_Sin(nn.Module):
    def __init__(self, num_classes, size):
        super(MultiModel_With_NonLinear_Sin, self).__init__()

        self.lr = 0.00001 # alpha
        self.M = torch.zeros(models, d).double() # Model initializes in 0
        self.project = embeddings.Projection(1, d).double() # 5 features, 10000 dimensions = hypervectors like weights?
        self.project.weight.data.normal_(0, 1) # Normal distributions mean=0.0, std=1.0
        self.bias = nn.parameter.Parameter(torch.empty(d), requires_grad=False)
        self.bias.data.uniform_(0, 2 * math.pi) # bias
        self.cluster = functional.random_hv(models, d)

    def encode(self, x): # encoding a value
        vector_hv = torch.zeros(d)
        for i in range(len(x)):
            x[i] = float(x[i])
            enc = self.project(torch.tensor([x[i]]))
            permute = functional.permute(enc, shifts=i)
            vector_hv = functional.bundle(vector_hv,permute)
        return functional.hard_quantize(vector_hv)

    def model_update(self, x, y): # update # y = no hv
        x = torch.reshape(x, (1,d))
        confidence = np.transpose(softmax(cos_similarity(self.cluster, x))) # Compare input with cluster
        model_result = F.linear(x, self.M).type(torch.FloatTensor)
        update = self.M + (float(self.lr) * float(y - F.linear(confidence, model_result)) * x) # Model + alpha*(Error)*(x)
        #update = update.mean(0) # Mean by columns
        self.M = update # New 
        # update cluster center?
        center = [num.item() for num in confidence[0]].index(max(confidence[0]).item())
        self.cluster[center] = self.cluster[center] + (1-max(confidence[0])) * x
        

    def forward(self, x):
        enc = torch.reshape(self.encode(x), (1,d))
        confidence = np.transpose(softmax(cos_similarity(self.cluster, enc))) # Compare input with cluster
        model_result = F.linear(enc, self.M).type(torch.FloatTensor)
        res = F.linear(confidence, model_result) # Multiply enc (x) * weights (Model) = Dot product
        return res # Return the resolutions
    
model = MultiModel_With_NonLinear_Sin(2, 40) # 1 class, 5
#model = model.to(device)

#mse = torchmetrics.MeanSquaredError()
    
# Model training

start_time = time.time()
#main()
#print("--- %s seconds ---" % (time.time() - start_time))

## TRIED ONE ITERATION

100%|██████████| 1/1 [04:03<00:00, 243.10s/it]

Training mean squared error of 0.024


In [11]:
with torch.no_grad(): # disabled gradient calculation because were doing it manually
    for _ in tqdm(range(1)): # Number of iterations for all the samples
        mse = torchmetrics.MeanSquaredError()
        for i in sets_training:
            pred = []
            samples = matrix_1_norm[:, i:i+40]
            labels = matrix_1_norm[:, i+40]
            #print(f"sample: {samples}, label:{labels}")
            #samples = samples.to(device) # pass sample and label (1 at a time)
            #labels = labels.to(device)
            
            for n in range(samples.shape[0]):
                label = torch.tensor(labels[n])
                sample = torch.tensor(samples[n, :])
                #print(x)
                #samples = 
                samples_hv = model.encode(sample) # Encode the inputs
                #print(f"sample_hv:{samples_hv}")
                model.model_update(samples_hv, label) # Pass input and label to train

                predictions_testing = model(sample) # Pass samples from test to model (forward function)
                pred.append(predictions_testing[0])
                #predictions_testing = predictions_testing
                #label = label
                mse.update(predictions_testing[0], label.unsqueeze(dim=0))
                #mse.update(predictions_testing.cpu(), label)

        print(f"Training mean squared error of {(mse.compute().item()):.3f}")

100%|██████████| 1/1 [04:04<00:00, 244.88s/it]

Training mean squared error of 0.022


In [12]:
pred_multihd = []
original_hd = []

with torch.no_grad():
    mse = torchmetrics.MeanSquaredError()
    for i in tqdm(sets_testing):
        pred = []
        samples = matrix_1_norm[:, i:i+40]
        labels = matrix_1_norm[:, i+40]
        for n in range(samples.shape[0]):
            label = torch.tensor(labels[n])
            sample = torch.tensor(samples[n, :])
            # Pass samples from test to model (forward function)
            predictions = model(sample)
            pred.append(predictions[0])
            # predictions = predictions * TARGET_STD + TARGET_MEAN # What is target
            # labels = labels * TARGET_STD + TARGET_MEAN
            mse.update(predictions[0], label.unsqueeze(dim=0))
            # dif_hd.append(np.absolute(label-predictions))
            pred_multihd.append(predictions)
            original_hd.append(label)

    print(
        f"Testing mean squared error of testing {(mse.compute().item()):.3f}")

100%|██████████| 7/7 [00:35<00:00,  5.05s/it]

Testing mean squared error of testing 0.039


## Mltimodal RegHD with just one cluster per prediction

In [14]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

# Note: this example requires the torchmetrics library: https://torchmetrics.readthedocs.io
import torchmetrics
from tqdm import tqdm

from torchhd import functional
from torchhd import embeddings
from torchhd import cos_similarity

from scipy.special import softmax

import time

d=10000
models = 8

# Model based on RegHD application for Single model regression -> No comparing which cluster
class MultiModel_With_NonLinear_Sin(nn.Module):
    def __init__(self, num_classes, size):
        super(MultiModel_With_NonLinear_Sin, self).__init__()

        self.lr = 0.00001 # alpha
        self.M = torch.zeros(models, d).double() # Model initializes in 0
        self.project = embeddings.Projection(size, d).double() # 5 features, 10000 dimensions = hypervectors like weights?
        self.project.weight.data.normal_(0, 1) # Normal distributions mean=0.0, std=1.0
        self.bias = nn.parameter.Parameter(torch.empty(d), requires_grad=False)
        self.bias.data.uniform_(0, 2 * math.pi) # bias
        self.cluster = functional.random_hv(models, d)

    def encode(self, x): # encoding a value
        for i in range(len(x)):
            x[i] = float(x[i])
        enc = self.project(x)
        sample_hv = torch.cos(enc + self.bias) * torch.sin(enc) 
        return functional.hard_quantize(sample_hv)

    def model_update(self, x, y): # update # y = no hv
        x = torch.reshape(x, (1,d))
        confidence = np.transpose(softmax(cos_similarity(self.cluster, x))) # Compare input with cluster
        center = [num.item() for num in confidence[0]].index(max(confidence[0]).item())
        update = self.M[center] + (float(self.lr) * float(y - F.linear(x, self.M[center])) * x) # Model + alpha*(Error)*(x)
        #update = update.mean(0) # Mean by columns
        self.M[center] = update # New 
        # update cluster center?
        
        self.cluster[center] = self.cluster[center] + (1-max(confidence[0])) * x
        

    def forward(self, x):
        enc = torch.reshape(self.encode(x), (1,d))
        confidence = np.transpose(softmax(cos_similarity(self.cluster, enc)))
        center = [num.item() for num in confidence[0]].index(max(confidence[0]).item())
        #confidence = np.transpose(softmax(cos_similarity(self.cluster, enc))) # Compare input with cluster
        res = F.linear(enc, self.M[center])
        #res = F.linear(confidence, model_result) # Multiply enc (x) * weights (Model) = Dot product
        return res # Return the resolutions
    
model = MultiModel_With_NonLinear_Sin(2, 40) # 1 class, 5
#model = model.to(device)

#mse = torchmetrics.MeanSquaredError()
    
# Model training

start_time = time.time()
#main()
#print("--- %s seconds ---" % (time.time() - start_time))

100%|██████████| 1/1 [01:23<00:00, 83.03s/it]

Training mean squared error of 0.022


In [19]:
with torch.no_grad(): # disabled gradient calculation because were doing it manually
    for _ in tqdm(range(1)): # Number of iterations for all the samples
        mse = torchmetrics.MeanSquaredError()
        for i in sets_training:
            pred = []
            samples = matrix_1_norm[:, i:i+40]
            labels = matrix_1_norm[:, i+40]
            #print(f"sample: {samples}, label:{labels}")
            #samples = samples.to(device) # pass sample and label (1 at a time)
            #labels = labels.to(device)
            
            for n in range(samples.shape[0]):
                label = torch.tensor(labels[n])
                sample = torch.tensor(samples[n, :])
                #print(x)
                #samples = 
                samples_hv = model.encode(sample) # Encode the inputs
                #print(f"sample_hv:{samples_hv}")
                model.model_update(samples_hv, label) # Pass input and label to train

                predictions_testing = model(sample) # Pass samples from test to model (forward function)
                pred.append(predictions_testing)
                #predictions_testing = predictions_testing
                #label = label
                mse.update(predictions_testing, label.unsqueeze(dim=0))
                #mse.update(predictions_testing.cpu(), label)

        print(f"Training mean squared error of {(mse.compute().item()):.3f}")

100%|██████████| 1/1 [01:21<00:00, 81.38s/it]

Training mean squared error of 0.015


In [20]:
pred_multihd = []
original_hd = []

with torch.no_grad():
    mse = torchmetrics.MeanSquaredError()
    for i in tqdm(sets_testing):
        pred = []
        samples = matrix_1_norm[:, i:i+40]
        labels = matrix_1_norm[:, i+40]
        for n in range(samples.shape[0]):
            label = torch.tensor(labels[n])
            sample = torch.tensor(samples[n, :])
            # Pass samples from test to model (forward function)
            predictions = model(sample)
            pred.append(predictions)
            # predictions = predictions * TARGET_STD + TARGET_MEAN # What is target
            # labels = labels * TARGET_STD + TARGET_MEAN
            mse.update(predictions, label.unsqueeze(dim=0))
            # dif_hd.append(np.absolute(label-predictions))
            pred_multihd.append(predictions)
            original_hd.append(label)

    print(
        f"Testing mean squared error of testing {(mse.compute().item()):.3f}")

100%|██████████| 7/7 [00:11<00:00,  1.57s/it]

Testing mean squared error of testing 0.027


## Let's combine both just for the sake of it

In [24]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

# Note: this example requires the torchmetrics library: https://torchmetrics.readthedocs.io
import torchmetrics
from tqdm import tqdm

from torchhd import functional
from torchhd import embeddings
from torchhd import cos_similarity

from scipy.special import softmax

import time

d=10000
models = 8

# Model based on RegHD application for Single model regression -> No comparing which cluster
class MultiModel_With_NonLinear_Sin(nn.Module):
    def __init__(self, num_classes, size):
        super(MultiModel_With_NonLinear_Sin, self).__init__()

        self.lr = 0.00001 # alpha
        self.M = torch.zeros(models, d).double() # Model initializes in 0
        self.project = embeddings.Projection(1, d).double() # 5 features, 10000 dimensions = hypervectors like weights?
        self.project.weight.data.normal_(0, 1) # Normal distributions mean=0.0, std=1.0
        self.bias = nn.parameter.Parameter(torch.empty(d), requires_grad=False)
        self.bias.data.uniform_(0, 2 * math.pi) # bias
        self.cluster = functional.random_hv(models, d)

    def encode(self, x): # encoding a value
        vector_hv = torch.zeros(d)
        for i in range(len(x)):
            x[i] = float(x[i])
            enc = self.project(torch.tensor([x[i]]))
            permute = functional.permute(enc, shifts=i)
            vector_hv = functional.bundle(vector_hv,permute)
        return functional.hard_quantize(vector_hv)

    def model_update(self, x, y): # update # y = no hv
        x = torch.reshape(x, (1,d))
        confidence = np.transpose(softmax(cos_similarity(self.cluster, x))) # Compare input with cluster
        center = [num.item() for num in confidence[0]].index(max(confidence[0]).item())
        print(center)
        update = self.M[center] + (float(self.lr) * float(y - F.linear(x, self.M[center])) * x) # Model + alpha*(Error)*(x)
        #update = update.mean(0) # Mean by columns
        self.M[center] = update # New 
        # update cluster center?
        
        self.cluster[center] = self.cluster[center] + (1-max(confidence[0])) * x
        

    def forward(self, x):
        enc = torch.reshape(self.encode(x), (1,d))
        confidence = np.transpose(softmax(cos_similarity(self.cluster, enc)))
        center = [num.item() for num in confidence[0]].index(max(confidence[0]).item())
        #confidence = np.transpose(softmax(cos_similarity(self.cluster, enc))) # Compare input with cluster
        res = F.linear(enc, self.M[center])
        #res = F.linear(confidence, model_result) # Multiply enc (x) * weights (Model) = Dot product
        return res # Return the resolutions
    
model = MultiModel_With_NonLinear_Sin(2, 40) # 1 class, 5
#model = model.to(device)

#mse = torchmetrics.MeanSquaredError()
    
# Model training

start_time = time.time()
#main()
#print("--- %s seconds ---" % (time.time() - start_time))

In [25]:
with torch.no_grad(): # disabled gradient calculation because were doing it manually
    for _ in tqdm(range(1)): # Number of iterations for all the samples
        mse = torchmetrics.MeanSquaredError()
        for i in sets_training:
            pred = []
            samples = matrix_1_norm[:, i:i+40]
            labels = matrix_1_norm[:, i+40]
            #print(f"sample: {samples}, label:{labels}")
            #samples = samples.to(device) # pass sample and label (1 at a time)
            #labels = labels.to(device)
            
            for n in range(samples.shape[0]):
                label = torch.tensor(labels[n])
                sample = torch.tensor(samples[n, :])
                #print(x)
                #samples = 
                samples_hv = model.encode(sample) # Encode the inputs
                #print(f"sample_hv:{samples_hv}")
                model.model_update(samples_hv, label) # Pass input and label to train

                predictions_testing = model(sample) # Pass samples from test to model (forward function)
                pred.append(predictions_testing)
                #predictions_testing = predictions_testing
                #label = label
                mse.update(predictions_testing, label.unsqueeze(dim=0))
                #mse.update(predictions_testing.cpu(), label)

        print(f"Training mean squared error of {(mse.compute().item()):.3f}")

  0%|          | 0/1 [00:00<?, ?it/s]

7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7


  0%|          | 0/1 [00:14<?, ?it/s]

7
7


KeyboardInterrupt: 

In [23]:
pred_multihd = []
original_hd = []

with torch.no_grad():
    mse = torchmetrics.MeanSquaredError()
    for i in tqdm(sets_testing):
        pred = []
        samples = matrix_1_norm[:, i:i+40]
        labels = matrix_1_norm[:, i+40]
        for n in range(samples.shape[0]):
            label = torch.tensor(labels[n])
            sample = torch.tensor(samples[n, :])
            # Pass samples from test to model (forward function)
            predictions = model(sample)
            pred.append(predictions)
            # predictions = predictions * TARGET_STD + TARGET_MEAN # What is target
            # labels = labels * TARGET_STD + TARGET_MEAN
            mse.update(predictions, label.unsqueeze(dim=0))
            # dif_hd.append(np.absolute(label-predictions))
            pred_multihd.append(predictions)
            original_hd.append(label)

    print(
        f"Testing mean squared error of testing {(mse.compute().item()):.3f}")

100%|██████████| 7/7 [00:35<00:00,  5.10s/it]

Testing mean squared error of testing 0.040


## Lets try to group he cluster by external factors only

#### Timeseries

In [60]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

# Note: this example requires the torchmetrics library: https://torchmetrics.readthedocs.io
import torchmetrics
from tqdm import tqdm

from torchhd import functional
from torchhd import embeddings
from torchhd import cos_similarity, bind

from scipy.special import softmax

import time

d = 10000
models = 8

# Model based on RegHD application for Single model regression -> No comparing which cluster
class MultiModel_With_Data(nn.Module):
    def __init__(self, num_classes, size):
        super(MultiModel_With_Data, self).__init__()

        self.lr = 0.00001 # alpha
        self.M = torch.zeros(models, d).double() # Model initializes in 0
        self.project = embeddings.Projection(size, d).double() # 5 features, 10000 dimensions = hypervectors like weights?
        self.project.weight.data.normal_(0, 1) # Normal distributions mean=0.0, std=1.0
        self.project_int = embeddings.Projection(1, d).float()# 5 features, 10000 dimensions = hypervectors like weights?
        self.project_int.weight.data.normal_(0, 1) # Normal distributions mean=0.0, std=1.0
        self.bias = nn.parameter.Parameter(torch.empty(d), requires_grad=False)
        self.bias.data.uniform_(0, 2 * math.pi) # bias
        self.cluster = functional.random_hv(models, d)

    def encode(self, x): # encoding a value
        try:
            for i in range(len(x)):
                x[i] = float(x[i])
            enc = self.project(x)
        except:
            enc = self.project_int(torch.tensor([x[0]]))            
        sample_hv = torch.cos(enc + self.bias) * torch.sin(enc) 
        return functional.hard_quantize(sample_hv)

    def model_update(self, x, y, ts): # update # y = no hv
        x = torch.reshape(x, (1,d))
        ts_enc = self.encode([ts])
        confidence = np.transpose(softmax(cos_similarity(self.cluster, ts_enc))) # Compare input with cluster
        center = [num.item() for num in confidence[0]].index(max(confidence[0]).item())
        update = self.M[center] + (float(self.lr) * float(y - F.linear(x, self.M[center])) * x) # Model + alpha*(Error)*(x)
        #update = update.mean(0) # Mean by columns
        self.M[center] = update # New 
        # update cluster center?
        
        self.cluster[center] = self.cluster[center] + (1-max(confidence[0])) * ts_enc
        

    def forward(self, x, ts):
        enc = torch.reshape(self.encode(x), (1,d))
        confidence = np.transpose(softmax(cos_similarity(self.cluster, self.encode([ts]))))
        center = [num.item() for num in confidence[0]].index(max(confidence[0]).item())
        #confidence = np.transpose(softmax(cos_similarity(self.cluster, enc))) # Compare input with cluster
        res = F.linear(enc, self.M[center])
        #res = F.linear(confidence, model_result) # Multiply enc (x) * weights (Model) = Dot product
        return res # Return the resolutions
    
model = MultiModel_With_Data(2, 40) # 1 class, 5

In [64]:
with torch.no_grad(): # disabled gradient calculation because were doing it manually
    for _ in tqdm(range(1)): # Number of iterations for all the samples
        mse = torchmetrics.MeanSquaredError()
        for i in sets_training:
            pred = []
            samples = matrix_1_norm[:, i:i+40]
            labels = matrix_1_norm[:, i+40]
            #print(f"sample: {samples}, label:{labels}")
            #samples = samples.to(device) # pass sample and label (1 at a time)
            #labels = labels.to(device)
            
            for n in range(samples.shape[0]):
                label = torch.tensor(labels[n])
                sample = torch.tensor(samples[n, :])
                #print(x)
                #samples = 
                samples_hv = model.encode(sample) # Encode the inputs
                #print(f"sample_hv:{samples_hv}")
                model.model_update(samples_hv, label, n) # Pass input and label to train

                predictions_testing = model(sample, n) # Pass samples from test to model (forward function)
                pred.append(predictions_testing)
                #predictions_testing = predictions_testing
                #label = label
                mse.update(predictions_testing, label.unsqueeze(dim=0))
                #mse.update(predictions_testing.cpu(), label)

        print(f"Training mean squared error of {(mse.compute().item()):.3f}")

100%|██████████| 1/1 [02:57<00:00, 177.03s/it]

Training mean squared error of 0.030


In [65]:
pred_multihd = []
original_hd = []

with torch.no_grad():
    mse = torchmetrics.MeanSquaredError()
    for i in tqdm(sets_testing):
        pred = []
        samples = matrix_1_norm[:, i:i+40]
        labels = matrix_1_norm[:, i+40]
        for n in range(samples.shape[0]):
            label = torch.tensor(labels[n])
            sample = torch.tensor(samples[n, :])
            # Pass samples from test to model (forward function)
            predictions = model(sample, n)
            pred.append(predictions)
            # predictions = predictions * TARGET_STD + TARGET_MEAN # What is target
            # labels = labels * TARGET_STD + TARGET_MEAN
            mse.update(predictions, label.unsqueeze(dim=0))
            # dif_hd.append(np.absolute(label-predictions))
            pred_multihd.append(predictions)
            original_hd.append(label)

    print(
        f"Testing mean squared error of testing {(mse.compute().item()):.3f}")

100%|██████████| 7/7 [00:23<00:00,  3.30s/it]

Testing mean squared error of testing 0.033


#### Timestamp

In [66]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

# Note: this example requires the torchmetrics library: https://torchmetrics.readthedocs.io
import torchmetrics
from tqdm import tqdm

from torchhd import functional
from torchhd import embeddings
from torchhd import cos_similarity, bind

from scipy.special import softmax

import time

d = 10000
models = 8

# Model based on RegHD application for Single model regression -> No comparing which cluster
class MultiModel_With_Data(nn.Module):
    def __init__(self, num_classes, size):
        super(MultiModel_With_Data, self).__init__()

        self.lr = 0.00001 # alpha
        self.M = torch.zeros(models, d).double() # Model initializes in 0
        self.project = embeddings.Projection(size, d).double() # 5 features, 10000 dimensions = hypervectors like weights?
        self.project.weight.data.normal_(0, 1) # Normal distributions mean=0.0, std=1.0
        self.project_int = embeddings.Projection(1, d).float()# 5 features, 10000 dimensions = hypervectors like weights?
        self.project_int.weight.data.normal_(0, 1) # Normal distributions mean=0.0, std=1.0
        self.bias = nn.parameter.Parameter(torch.empty(d), requires_grad=False)
        self.bias.data.uniform_(0, 2 * math.pi) # bias
        self.cluster = functional.random_hv(models, d)

    def encode(self, x): # encoding a value
        try:
            for i in range(len(x)):
                x[i] = float(x[i])
            enc = self.project(x)
        except:
            enc = self.project_int(torch.tensor([x[0]]))            
        sample_hv = torch.cos(enc + self.bias) * torch.sin(enc) 
        return functional.hard_quantize(sample_hv)

    def model_update(self, x, y, ts): # update # y = no hv
        x = torch.reshape(x, (1,d))
        ts_enc = self.encode([ts])
        confidence = np.transpose(softmax(cos_similarity(self.cluster, ts_enc))) # Compare input with cluster
        center = [num.item() for num in confidence[0]].index(max(confidence[0]).item())
        update = self.M[center] + (float(self.lr) * float(y - F.linear(x, self.M[center])) * x) # Model + alpha*(Error)*(x)
        #update = update.mean(0) # Mean by columns
        self.M[center] = update # New 
        # update cluster center?
        
        self.cluster[center] = self.cluster[center] + (1-max(confidence[0])) * ts_enc
        

    def forward(self, x, ts):
        enc = torch.reshape(self.encode(x), (1,d))
        confidence = np.transpose(softmax(cos_similarity(self.cluster, self.encode([ts]))))
        center = [num.item() for num in confidence[0]].index(max(confidence[0]).item())
        #confidence = np.transpose(softmax(cos_similarity(self.cluster, enc))) # Compare input with cluster
        res = F.linear(enc, self.M[center])
        #res = F.linear(confidence, model_result) # Multiply enc (x) * weights (Model) = Dot product
        return res # Return the resolutions
    
model = MultiModel_With_Data(2, 40) # 1 class, 5

In [71]:
with torch.no_grad(): # disabled gradient calculation because were doing it manually
    for _ in tqdm(range(1)): # Number of iterations for all the samples
        mse = torchmetrics.MeanSquaredError()
        for i in sets_training:
            pred = []
            samples = matrix_1_norm[:, i:i+40]
            labels = matrix_1_norm[:, i+40]
            #print(f"sample: {samples}, label:{labels}")
            #samples = samples.to(device) # pass sample and label (1 at a time)
            #labels = labels.to(device)
            
            for n in range(samples.shape[0]):
                label = torch.tensor(labels[n])
                sample = torch.tensor(samples[n, :])
                #print(x)
                #samples = 
                samples_hv = model.encode(sample) # Encode the inputs
                #print(f"sample_hv:{samples_hv}")
                model.model_update(samples_hv, label, i) # Pass input and label to train

                predictions_testing = model(sample, i) # Pass samples from test to model (forward function)
                pred.append(predictions_testing)
                #predictions_testing = predictions_testing
                #label = label
                mse.update(predictions_testing, label.unsqueeze(dim=0))
                #mse.update(predictions_testing.cpu(), label)

        print(f"Training mean squared error of {(mse.compute().item()):.3f}")

100%|██████████| 1/1 [02:53<00:00, 173.19s/it]

Training mean squared error of 0.028


In [72]:
pred_multihd = []
original_hd = []

with torch.no_grad():
    mse = torchmetrics.MeanSquaredError()
    for i in tqdm(sets_testing):
        pred = []
        samples = matrix_1_norm[:, i:i+40]
        labels = matrix_1_norm[:, i+40]
        for n in range(samples.shape[0]):
            label = torch.tensor(labels[n])
            sample = torch.tensor(samples[n, :])
            # Pass samples from test to model (forward function)
            predictions = model(sample, i)
            pred.append(predictions)
            # predictions = predictions * TARGET_STD + TARGET_MEAN # What is target
            # labels = labels * TARGET_STD + TARGET_MEAN
            mse.update(predictions, label.unsqueeze(dim=0))
            # dif_hd.append(np.absolute(label-predictions))
            pred_multihd.append(predictions)
            original_hd.append(label)

    print(
        f"Testing mean squared error of testing {(mse.compute().item()):.3f}")

100%|██████████| 7/7 [00:30<00:00,  4.36s/it]

Testing mean squared error of testing 0.032
